# Topic Modelling

From someone who does not understand Machine Learning.

In [1]:
import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LsiModel
from gensim.similarities import MatrixSimilarity

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from pprint import pprint

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

### Utils

#### Serialize

In [ ]:
import pickle


def pickle_save(obj, path):
     with open(path, 'wb') as out:
        pickle.dump(obj, out, protocol=pickle.HIGHEST_PROTOCOL)
            

def pickle_load(path):
     with open(path, 'rb') as obj:
        return pickle.load(obj)

#### Text Processing

In [ ]:
import re


def _remove_html_tags(text):
    cleaner = re.compile('<.*?>')
    return re.sub(cleaner, '', text)


def _remove_control_chars(text):
    text = text.replace('\n','')
    text = text.replace('\t','')
    return text


def _remove_stop_words(text):
    stop_words = list()
    stop_words.extend(stopwords.words('english'))

    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)
    result = []
    for word in words:
        if word not in stop_words:
            result.append(word)
    return " ".join(list(result))


def clean_text(text):
    text = _remove_html_tags(text)
    text = text.lower()
    text = _remove_control_chars(text)
    text = _remove_stop_words(text)
    return text

#### Gather Data

In [ ]:
import pylast
import settings


client = pylast.LastFMNetwork(api_key=settings.API_KEY,
                              api_secret=settings.API_SECRET,
                              username=settings.USER,
                              password_hash=pylast.md5(settings.PASSWORD))


def get_artist_bio(artist_name: str):
    try:
        artist = client.get_artist(artist_name)
        bio = artist.get_bio_content()

        if bio and 'There are at least' not in bio:  # No perfect match found
            return bio
    except pylast.WSError:
        print(f"Bio for {artist_name} cannot be found.")
    except pylast.MalformedResponseError:
        print(f"Bio for {artist_name} cannot be found.")

# Buckle Up!

In [ ]:
FROM_SCRATCH = True
INPUT_DATA_PATH = 'data/data.csv'
MODEL_PATH = 'model.pickle'

In [ ]:
!tree -I venv # With ! you can run bash commands in Jupyter

## Data Cleansing and Preparation

We create so called documents that consist of the artist name and their biography in tokenized format.

The mapping looks something like this

```
{
    1: {'artist_name': '50 Cent', 'bio': ['a', 'text', 'about', '50', 'Cent'},
    2: {'artist_name': 'Eminem', 'bio': ['a', 'text', 'about', 'Eminem'},
    3: {'artist_name': 'Metallica', 'bio': ['a', 'text', 'about', 'Metallica'},
    ...
}
```

### Input Data

In [ ]:
import csv


def load_csv(path: str) -> list:
    with open(path, 'r', newline='') as f:
        csv_reader = csv.reader(f, delimiter=',', quotechar='"')
        return [row for row in csv_reader]

    
rows = load_csv(INPUT_DATA_PATH)

In [ ]:
pprint(rows[2727])

In [ ]:
mapping = {}
for doc_number, r in enumerate(rows, 1):
    mapping[doc_number] = {
        'artist_name': r[0], 
        'bio': clean_text(r[1])
    }

In [ ]:
pprint(mapping[2728])

## Create model

#### <Machine Learning Magic World 🧙‍♂️>

The following code is not there to be understood. Thanks. `¯\_(ツ)_/¯`

In [ ]:
from dataclasses import dataclass


@dataclass
class MyModel:
    dictionary: Dictionary
    tfidf_model: TfidfModel
    lsi_model: LsiModel
    index: MatrixSimilarity
    

if FROM_SCRATCH:
    dataset = [mapping[i]['bio'] for i in mapping]
    
    dct = Dictionary(dataset)

    corpus = [dct.doc2bow(line) for line in dataset]
    tfidf_model = TfidfModel(corpus)
    tfidf_corpus = tfidf_model[corpus]

    lsi_model = LsiModel(tfidf_corpus, id2word=dct, num_topics=15, power_iters=4)

    index = MatrixSimilarity(lsi_model[tfidf_corpus])
    my_model = MyModel(dictionary=dct,
                       tfidf_model=tfidf_model,
                       lsi_model=lsi_model,
                       index=index)
    pickle_save(my_model, MODEL_PATH)
else:
    my_model = pickle_load(MODEL_PATH)




#### </Machine Learning Magic World 🧙‍♂️>

## Query the Model

Create the query entity data as we did when creating the model

In [ ]:
query_bio = clean_text(get_artist_bio('Britney Spears'))

#### <Machine Learning Magic World 🧙‍♂️>

The following code is not there to be understood. Thanks. `¯\_(ツ)_/¯`

In [ ]:
vec_bow = my_model.dictionary.doc2bow(query_bio)
vec_lsi = my_model.lsi_model[my_model.tfidf_model[vec_bow]]
similar_entities = my_model.index[vec_lsi]

#### </Machine Learning Magic World 🧙‍♂️>

## The Result

We got all the doc numbers (see `mapping`) and how similar they are to the queried artist (0.0 to 1.0). 

In [ ]:
doc_to_similarity = dict(enumerate(similar_entities))
pprint(doc_to_similarity)

Now let's find the top 10 similar artists and bands.

In [ ]:
top_n_docs = sorted(doc_to_similarity, key=doc_to_similarity.get, reverse=True)[:10]

result = []
for doc_number in top_n_docs:
    result.append({
        'similarity': similar_entities[doc_number],
        'artist_name': mapping[doc_number]['artist_name']
    })
    
pprint(result)

## Demo

### 🤖 `I am a semi professional ML implementation AMA`

In [ ]:
def find_similar_artists(artist_name):
    # <Machine Learning Magic World 🧙‍♂️>
    query_bio = clean_text(get_artist_bio(artist_name))
    vec_bow = my_model.dictionary.doc2bow(query_bio)
    vec_lsi = my_model.lsi_model[my_model.tfidf_model[vec_bow]]
    similar_entities = my_model.index[vec_lsi]
    # </Machine Learning Magic World 🧙‍♂️>
    doc_to_similarity = dict(enumerate(similar_entities))
    top_n_docs = sorted(doc_to_similarity, key=doc_to_similarity.get, reverse=True)[:10]

    result = []
    for doc_number in top_n_docs:
        result.append({
            'similarity': similar_entities[doc_number],
            'artist_name': mapping[doc_number]['artist_name']
        })
    sorted(result, key=lambda d: d['similarity'])
    pprint(result)

In [ ]:
find_similar_artists("Eminem")

In [ ]:
def find_similar_artists_by_prose(prose: str):
    clean_prose = clean_text(prose)
    # <Machine Learning Magic World 🧙‍♂️>
    vec_bow = my_model.dictionary.doc2bow(clean_prose)
    vec_lsi = my_model.lsi_model[my_model.tfidf_model[vec_bow]]
    similar_entities = my_model.index[vec_lsi]
    # </Machine Learning Magic World 🧙‍♂️>
    doc_to_similarity = dict(enumerate(similar_entities))
    top_n_docs = sorted(doc_to_similarity, key=doc_to_similarity.get, reverse=True)[:10]

    result = []
    for doc_number in top_n_docs:
        result.append({
            'similarity': similar_entities[doc_number],
            'artist_name': mapping[doc_number]['artist_name']
        })
    sorted(result, key=lambda d: d['similarity'])
    pprint(result)

In [ ]:
prose = "old school rap music from new york"
find_similar_artists_by_prose(prose)

## Open Questions

* How to update model without reading all data again?
* Do too many artists from one genre sabotage the result?


## Problems That I faced

* I did not save artists without bios. So everytime my data gathering broke, the rerunning script had to ask for all those artists again. 
* Artists that were duplicates but with uppercase/lowercase could not be added again to git index. Git does not seem to be the best blob storage.